# Import libraries

In [ ]:
import pandas as pd

# Read datasets

In [ ]:
ui_data = pd.read_csv('https://startg2545.github.io/user_item_tutorial.csv')
i_data = pd.read_csv('https://startg2545.github.io/item_tutorial.csv')

# Evaluation

In [ ]:
from src.isne_recommendation import TfidfLinearKernel

In [ ]:
model = TfidfLinearKernel.fit(i_data)
prediction = TfidfLinearKernel.predict('Martha Long', i_data, ui_data, model, 10)

In [ ]:
train, test = TfidfLinearKernel.train_test_split(ui_data)
evaluation = TfidfLinearKernel.evaluate_model(train, test, i_data, model)
evaluation

In [ ]:
from isne_recommendation import FeatureRatingsKNN

In [ ]:
model = FeatureRatingsKNN.fit(ui_data)
prediction = FeatureRatingsKNN.predict('Martha Long', ui_data, model, 10)

In [ ]:
train, test = FeatureRatingsKNN.train_test_split(ui_data)
model = FeatureRatingsKNN.fit(train)
evaluation = FeatureRatingsKNN.evaluate_model(train, test, model)
evaluation

In [ ]:
from src.isne_recommendation import Hybrid

In [ ]:
model = Hybrid.fit(i_data, ui_data)
prediction = Hybrid.predict('Martha Long', i_data, ui_data, model, 10)

In [ ]:
train, test = Hybrid.train_test_split(ui_data)
model = Hybrid.fit(i_data, train)
evaluation = Hybrid.evaluate_model(train, test, i_data, model)
evaluation

### Results 

- **The first approach**: _TF-IDF_ and _Linear Kernel_ Performance has an accuracy score of 13.70 percentage
- **The second approach**: _Feature Ratings_ and _KNN Performance_ has an accuracy score of 43.50 percentage
- **The third approach**: _Hybrid_ has an accuracy score of 75.68 percentage